# notebooks/main.ipynb
**Mục tiêu:** đây là notebook entry-point (demo / quick-start) cho repo.
Nội dung hướng dẫn cách chạy lần lượt các script trong `src/` (nhớ rằng các file `.py` của bạn đều nằm trong `src/` và `config.py` đang ở ngoài `src/` ở thư mục gốc của repo).
**Ghi chú:** chạy các ô theo thứ tự; một vài ô gọi trực tiếp các script trong `src/` (bằng lệnh `!python ...`), ô "Quick demo" sẽ chạy một bản demo nhẹ (không thay thế cho full-training scripts trong `src/`).

In [ ]:
# 1) Setup môi trường trong notebook (chạy từ thư mục notebooks/)
import os, sys
# Chuyển cwd về repo root (giả sử notebook nằm trong notebooks/)
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print('Detected repo_root:', repo_root)
os.chdir(repo_root)
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print('Current working directory:', os.getcwd())
print('Python sys.path[0] =', sys.path[0])

# Nếu bạn chạy notebook trực tiếp từ repo root, ô trên vẫn an toàn.


## 2) (Tùy chọn) Cài dependencies
Nếu bạn dùng Colab hoặc môi trường mới, hãy uncomment và sửa `requirements.txt` hoặc cài tay các package cần thiết. Các script trong `src/` dùng: `pandas`, `numpy`, `sentence-transformers`, `xgboost`, `scikit-learn`, `joblib`, `matplotlib`, `seaborn`, `nltk`, `torch`...
Bạn có thể chạy cell bên dưới để cài nhanh (nếu cần).

In [ ]:
# !pip install -r requirements.txt
# ví dụ (bỏ dấu # để chạy):
# !pip install pandas numpy sentence-transformers xgboost scikit-learn joblib matplotlib seaborn nltk torch tqdm

# Nếu bạn dùng Colab và muốn mount Google Drive, thực hiện ở đây.


## 3) Kiểm tra GPU (với script có sẵn)
Chạy file `src/GPU_check.py` để biết GPU có sẵn hay không.

In [ ]:
# Chạy script kiểm tra GPU
!python src/GPU_check.py


## 4) Data cleaning (script `2.1_data_clean.py`)
Chạy script dọn dữ liệu — nó đọc `config.RAW_DATA_PATH` và lưu `config.CLEAN_DATA_PATH`.

In [ ]:
# Chạy data cleaning (lưu ý: config.py phải trỏ đúng đường dẫn ở trên máy bạn)
!python src/2.1_data_clean.py


## 5) Data augmentation (script `2.2_data_augmentation.py`)
Script này sẽ đọc `config.CLEAN_DATA_PATH` và ghi `config.FINAL_DATA_PATH`.
**Ghi chú:** script dùng `nltk` để augment — nếu chưa có, nó sẽ cố tải các gói cần thiết (có thể mất thời gian).

In [ ]:
# Chạy augmentation
!python src/2.2_data_augmentation.py


## 6) Khám phá nhanh dữ liệu (script `1_view_data.py`)
Script sẽ tạo các ảnh EDA (saves .png) để bạn kiểm tra phân phối nhãn và độ dài văn bản.

In [ ]:
# Chạy EDA view
!python src/1_view_data.py
print('EDA images saved to current working directory (look for PNG files).')


## 7) Quick demo: tính embedding + train XGBoost nhỏ (chạy nhanh, dùng để kiểm tra pipeline)
**Lưu ý:** Đây là bản demo rút gọn để bạn test notebook — không thay thế scripts full training trong `src/` (3.1 / 3.2).
Nếu muốn chạy full training, dùng `!python src/3.2_training_multi.py` hoặc `!python src/3.1_training_binary.py` theo nhu cầu.

In [ ]:
# Quick demo: sample nhỏ, encode bằng SentenceTransformer và train XGBoost nhẹ
import config
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Đọc file đã xử lý (do 2.2 tạo ra)
df = pd.read_csv(config.FINAL_DATA_PATH).dropna(subset=['type','posts'])
n_samples = min(400, len(df))
print(f'Dataset size after augment: {len(df)}; using {n_samples} rows for quick demo')
df = df.sample(n=n_samples, random_state=42)
X_texts = df['posts'].astype(str).tolist()
y = df['type'].astype('category').cat.codes.values
label_names = list(df['type'].astype('category').cat.categories)

# Load embedder (device sẽ tự chọn; nếu muốn ép dùng CPU/GPU, sửa device)
device = 'cuda' if __import__('torch').cuda.is_available() else 'cpu'
print('Using device for embedder:', device)
embedder = SentenceTransformer(config.MODEL_NAME, device=device)

def encode_posts(posts):
    parts = posts.split('|||')
    emb = embedder.encode(parts, convert_to_numpy=True, batch_size=config.BATCH_SIZE, show_progress_bar=False)
    return emb.mean(axis=0)

# Encode (this may take a little time depending on samples)
X_emb = np.array([encode_posts(p) for p in X_texts])

# Quick train/test split and a tiny XGBoost (lightweight)
X_train, X_test, y_train, y_test = train_test_split(X_emb, y, test_size=0.2, random_state=42, stratify=y)
clf = xgb.XGBClassifier(objective='multi:softmax', num_class=len(label_names), n_estimators=50, max_depth=4, use_label_encoder=False, eval_metric='mlogloss')
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print('Quick demo accuracy:', accuracy_score(y_test, pred))


## 8) (Tuỳ chọn) Chạy full training hoặc evaluation scripts từ `src/`
Nếu bạn muốn chạy full multiclass training (cẩn thận: lâu, cần nhiều tài nguyên):
```bash
# multiclass full training (chạy từ repo root)
python src/3.2_training_multi.py
# binary training (4 models)
python src/3.1_training_binary.py
# evaluation scripts
python src/4.1_evaBIN.py
python src/4.2_evaMUL.py
```
Các script sẽ lưu kết quả vào các folder được định nghĩa trong `config.py` (`OUTPUT_DIR_BINARY`, `OUTPUT_DIR_MULTICLASS`).

## 9) Lời khuyên khi dùng notebook này
- Đảm bảo `config.py` trỏ đúng đường dẫn dữ liệu (`RAW_DATA_PATH`, `CLEAN_DATA_PATH`, `FINAL_DATA_PATH`).
- Nếu chạy trên Colab: mount Google Drive và sửa `config.py` hoặc set các biến môi trường tương ứng.
- Full training cần GPU và thời gian — dùng `Quick demo` để kiểm tra pipeline trước.
- Nếu bạn muốn mình convert notebook này thành file `.ipynb` tải về trực tiếp, báo mình — mình sẽ xuất file để bạn download.
